<a href="https://colab.research.google.com/github/hi-rama/AI-Machine-Learning-Study/blob/main/263p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

캐글의 신용카드 데이터 세트를 이용해 신용카드 사기 검출 분류 실습을 수행해 보겠습니다. 

레이블이 불균형한 분포를 가진 데이터 세트를 학습시킬 때 예측 성능의 문제가 발생하 수 있는데 이는 이상 레이블을 가지는 데이터 건수가 정상 레이블을 가진 데이터 건수에 비해 너무 적기 때문에 발생합니다. 즉 이상 레이블을 가지는 데이터 건수는 매우 적기 때문에 제대로 다양한 유형을 학습하지 못하는 반면에 정상 레이블을 가지는 데이터 건수는 매우 많기 때문에 일방적으로 정상 레이블로 치우친 학습을 제대로 된 이상 데이터 검출이 어려워지기 쉽습니다. 

지도 학습에서 극도로 불균형한 레이블 값 분포로 인한 문제점을 해결하기 위해서는 적절한 학습 데이터를 확보하는 방안이 필요한데, 대표적으로 오버 샘플링과 언더 샘플링이 있으며 오버 샘플링 방식이 예측 성능상 더 유리한 경우가 많아 주로 사용됩니다. 

오버 샘플링은 이상 데이터와 같이 적은 데이터를 증식하여 학습을 위한 충분한 데이터를 확보하는 방법입니다. 동일한 데이터를 단순히 증식하지 않고 원본 데이터의 값을 아주 약간만 변경하여 증식합니다.

smote 방법은 적은 데이터 세트에 있는 개별 데이터들의 K최근접 이웃을 찾아서 이 데이터와 K개 이웃들의 차이를 일정 값으로 만들어서 기존 데이터와 약간 차이가 나는 새로운 데이터들을 생성하는 방식입니다.

smote를 구현한 대표적인 파이썬 패키지는 imbalanced-learn입니다. 

In [ ]:
conda install -c conda-forge imbalanced-learn


데이터 1차 가공 및 모델 학습/예측/평가

설치가 됐으면 이제 데이터 세트를 로딩하고 모델을 생성하겠습니다. 


파일을 동일한 디렉토리로 이동시킨 후 DataFrame으로 로딩합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

card_df = pd.read_csv('./creditcard.csv')
card_df.head(3)

creditcard.csv의 v로 시작하는 feature들의 의미는 알 수가 없습니다. time피처의 경우는 데이터 생성 관련한 작업용 속성으로서 큰 의미가 없기에 제거하겠습니다. 
Amount feature 는 신용카드 트랙잭션 금액을 의미하며, Class는 레이블로서 0의 경우 정사상, 1의 경우 사기 트랙잭션입니다. card_df.info()로 확인해 보면 Missing value는 없으며, Class 레이블만 int형이고 나머지는 float형입니다.

이번 실습에서는 보다 다양한 데이터 전처리를 수행하고 이에따른 예측 성능도 함께 비교해 보겠습니다. 이를 위해 DataFrame을 복사한 뒤, 이를 가공하여 반환하는 get_preprocessed_df() 함수와 데이터 가공 후 학습/테스트 데이터 세트를 반환하는 get_train_test_df() 함수를 생성하겠습니다. 먼저 get_preprocessed_df() 함수는 불필요한 Time feature만 삭제하는 것으로 시작하겠습니다.

In [ ]:
from sklearn.model_selection import train_test_split

# 인자로 입력받은 DataFrame을 복사한 뒤 Time 칼럼만 삭제하고 복사된 DataFrame 반환
def get_preprocessed_df(df=None):
  df_copy=df.copy()
  df_copy.drop('Time',axis=1,inplace=True)
  return df_copy


get_train_test_dataset()는 get_preprocessed_df()를 호출한 뒤 데이터 세트를 구분합니다. get_train_test_dataset()는 내부에서 train_test_split() 함수를 호출하며, 테스트 데이터 세트를 전체의 30%인 Stratified 방식으로 추출해 학습 데이터 세트와 테스트 데이터 세트의 레이블 값 분포도를 서로 동일하게 만듭니다.

In [ ]:
def get_train_test_dataset(df=None):
  #인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환
  df_copy = get_preprocessed_df(df)
  X_features = df_copy.iloc[:,:-1]
  y_target=df_copy.iloc[:,-1]

  X_train,X_test,y_train,y_test = \
  train_test_split(X_features, y_target, test_size=0.3, random_state=0, stratify=y_target)
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)

데이터가 잘 분할됐습니다. 이제 모델을 만들어 보겠습니다. 로지스틱 회귀와 LightGBM 기반의 모델이 데이터 가공을 수행하면서 예측 성능이 어떻게 변하는지 살펴 볼 것입니다. 먼저 로지스틱 회귀를 이용해 신용 카드 사기 여부를 예측해 보겠습니다. 

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train,y_train)
lr_pred = lr_clf.predict(X_test)
lr_pred_proba = lr_clf.predict_proba(X_test)[:,1]


이번에는 LightGBM을 이용한 모델을 만들어 보겠습니다. 그에 앞서, 앞으로 수행할 예제 코드에서 반복적으로 모델을 변경해 학습/예측/평가할 것이므로 이를 위한 별도의 함수를 생성하겠습니다. get_moddel_train_eval()는 인자로 사이킷런의 Estimator 객체와 학습/데이터 셋을 입력 받아서 학습과 예측 평가를 수행합니다.

In [ ]:
def get_model_train_eval(model,ftr_train=None,ftr_test=None,tgt_train=None,tgt_test=None):
  model.fit(ftr_train,tgt_train)
  pred = model.predict(ftr_test)
  pred_proba = model.predict_proba(ftr_test)[:,1]
 # get_clf_eval(tgt_test,pred,pred_proba)


LightGBM 으로 모델을 학습시킨 뒤, 별도의 테스트 데이터 세트에서 예측 평가를 수행해 보겠습니다.

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf,ftr_train=X_train,ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

데이터 분포도 변환 후 모델 학습

이번에는 왜곡된 분포도를 가지는 데이터를 재가공한 뒤에 모델을 다시 테스트해 보겠습니다. 먼저 creditcard.csv의 중요 피처 값의 분포도를 살펴봅시다. 

In [ ]:
import seaborn as sns
plt.figure(figsize=(8,4))
plt.xticks(range(0,3000,100)),rotation=60)
sns.distplot(card_df['Amount'])

In [ ]:
from sklearn.preprocessing import StandardScaler

def get_preprocessed_df(df=None):
  df_copy = df.copy()
  scaler = StandardScaler()
  amount_n = scaler.fit_transform(df_copy['Amount'].values.reshape(-1,1))
  df_copy,insert(0,'Amount_Scaled',amount_n)
  df_copy.drop(['Time','Amount'], axis=1, inplace=True)
  return df_copy

get_train_test_dataset()을 호출해 데이터 세트를 생성한 후에 get_model_train_eval()을 이용해 로지스틱 회귀와 LightGBM 모델을 각각 학습해 보겠ㅅ브니다.

정규 분포 형태로 amount feature를 변환한 후의 모델은 성능이 크게 개선되지 않았습ㄴ니ㅏㄷ.

이번에는 로그 변환을 수행해 보겠습니다. 로그 변환은 데이터 분포도가 심하게 왜곡돼 있으면 적용하는 기법 중에 하나입니다. 넘파이의 log1p()함수를 이용해 간단히 변환이 가능합니다. 데이터 가공 함수인 get_preprocessed_df()를 다음과 같이 변경합니다

In [ ]:
def get_preprocessed_df(df=None):
  df_copy = df.copy()
  amount_n = np.log1p(df_copy['Amoun'])
  df_copy.insert(0,'Amount_Scaled',amount_n)
  df_copy.drop(['Time','Amount'],axis=1, inplace=True)
  return df_copy


이상치 데이터 제거 (Outlier)

이상치를 찾는 방법 중 IQR을 적용하겠ㅅ브니다.
먼저 어떤 feature의 이상치 데이터를 검출할 것인지 선택이 필요합니다. 레이블과 가장 상관성이 높은 feature들을 위주로 이상치를 검출하는 것이 좋습니다. DataFrame의 corr()을 이용해 각 피처별로 상관도를 구한 뒤 heatmap을 통해 시각화해 보겠습니ㅏㄷ.

In [ ]:
import seaborn as sns

plt.figure(figsize=(9,9))
corr = card_df.corr()
sns.heatmap(corr,cmap='RdBu')

rufwjd fpdlqmfrhk tkdrhksrhksrPrk shvdms 레이블에 대해서만 이상치를 찾아서 제거해 보겠습니다. IQR을 이용해 이상치를 검출하는 함수를 생성한 뒤, 이를 이용해 이상치를 삭제합니다.

GET_OUTLIER() 함수는 인자로 DF와 이상치를 검출한 칼럼을 입력받습니다. 넘파이의 percentline()을 이용해 1/4 분위와 3/4 분위를 구하고, IQR를 계산합니다.
계산된 IQR에 1.5를 곱해서 최댓값과 최솟값의 지점을 구한 뒤, 최댓값보다 크거나 최솟값보다 작은 값을 이상치로 설정하고 해당 이상치가 있는 DF index를 반환합니다.

In [ ]:
import numpy as np

def get_outlier(df=None, column=None, weight=1.5):
  fraud= df[df['Class']=1][column]
  quantile_25 = np.percentlie(fraud.values,25)
  quantile_75 = np.percentlie(fraud.values,75)

  iqr = quantile_75 - quantlie_25
  iqr_weight = iqr*weight
  lowest_val = quantlie_25 - iqr_weight
  highest_val = quantlie_75 + iqr_weight
  outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
  return outlier_index

In [ ]:
outlier_index = get_outlier(df=card_df, column='V14', weight = 1.5)


SMOTE 오버 샘플링 적용 후 모델 학습 예측 평가

SMOTE는 imbalanced-learn 패키지의 SMOTE 클래스를 이용해 간단하게 구현이 가능합니다. SMOTE를 적용할 땐느 학습 데이터만 오버 새뮤플링해야 합니다.

fit_sample() 메서드를 이용해 증식한 뒤 데이터 증식 전과 비교해 보겠습니다.

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)
X_train,y_train_over = smote.fit_sample(X_train, y_train)
